# Getting Game IDs

In [51]:
import requests
from bs4 import BeautifulSoup

url = "https://fbref.com/en/comps/10/2022-2023/schedule/2022-2023-Championship-Scores-and-Fixtures"

# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page
soup = BeautifulSoup(response.text, 'html.parser')

# Find all <td> elements with data-stat="score"
score_cells = soup.find_all('td', {'data-stat': 'score'})

# Extract game IDs from the <a> tags within the score_cells, the third element is the game ID
game_ids = [cell.find('a')['href'].split('/')[3] for cell in score_cells if cell.find('a')]
# Then strip the white space in the IDs
game_ids = [x.strip(' ') for x in game_ids]

In [52]:
game_ids = game_ids[0:3]

# Scraping Shot Data

In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

all_data = []
for unique_id in game_ids:
    # URL of the page to scrape
    url = f'https://fbref.com/en/matches/{unique_id}/'

    # Get the game ID from the URL
    game_id = url.split("/")[-2]

    # Send a GET request to the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the Shots table on the page
    shots_table = soup.find("div", {"id": "switcher_shots"})

    # Create empty lists to store data and create the columns
    data = []
    headers = ['Game_ID', 'Minute', 'Player', 'Team', 'xG', 'PSxG', 'Result', 'Distance', 'Body Part', 'Notes', 'SCA 1 Player', 'Event 1', 'SCA 2 Player', 'Event 2']

    # Extract table rows
    rows = shots_table.find("tbody").find_all("tr")
    for row in rows:
        row_data = [game_id] + [cell.get_text().strip() for cell in row.find_all(["th", "td"])]
        data.append(row_data)

    # Create a DataFrame from the scraped data
    shots_df = pd.DataFrame(data, columns=headers)

    # Find the home and away teams and formations
    team_headers = soup.find_all("th", {"colspan": "2"})
    teams = [header.text.strip().split(" (")[0] for header in team_headers if "(" in header.text.strip()]
    formations = [header.text.strip().split(" (")[1][:-1] for header in team_headers if "(" in header.text.strip()]

    # Extracting the home and away teams and formations
    home_team = teams[0]
    away_team = teams[1]
    home_formation = formations[0]
    away_formation = formations[1]

    # Find the competition -- only using .find because there are multiple in the HTML and we only need one, then take only the text
    competition = soup.find("a", {"href": "/en/comps/10/2022-2023/2022-2023-Championship-Stats"}).text

    # Extract the match date and add to df, this one is a little tricky as it's hidden within the span tag
    span_venuetime = soup.find("span", class_="venuetime")
    match_date = span_venuetime.get("data-venue-date") if span_venuetime else None

    #Get location, 4th element in that list
    location = soup.find("div", {"class": "scorebox_meta"}).find_all("small")[3].text
    
    all_data.append(data)

In [55]:
#Flatten list one level to load into df
all_data = sum(all_data, [])

In [56]:
#There is one row in each shots table on FBREF that is completely blank, so we can remove that before we load lists into the df
#This logic says if the minute, player, and team columns are blank (doing three to be safe), we can drop the record
for i in all_data:
    if i[1] == '' and i[2] == '' and i[3] == '':
        all_data.remove(i)
    else:
        pass

In [57]:
# Create a DataFrame from the scraped data
shots_df = pd.DataFrame(all_data, columns=headers)

# Add the team, away team, formation, and opponent formation information to the DataFrame
shots_df['Home_Team'] = home_team
shots_df['Away_Team'] = away_team
shots_df['Home Formation'] = home_formation
shots_df['Away Formation'] = away_formation
shots_df['is_home_shot'] = np.where(shots_df['Team'] == home_team, 1, 0)
shots_df['is_away_shot'] = np.where(shots_df['Team'] ==  away_team, 1, 0)
shots_df['location'] = location
shots_df['match_date'] = match_date

In [59]:
#other leagues